<a href="https://colab.research.google.com/github/pmadhyastha/INM434/blob/main/ngram_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import Math, HTML
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
               "latest.js?config=default'></script>"))

#Language models

As we noted in the lecture today, language models are models that are trained to predict the likelihood of a sequence of words or characters in a language. The primary goal of language models is to learn the structure and patterns of a language by analyzing large amounts of text. 


We will begin our exploration by looking at the key fundamentals of the simplest language model. 

We will start of with a toy passage from Simple English Wikipedia. 

In [ ]:
# our toy text (ideally we would train over large amounto of data, this is just a tiny snapshot)
input_text = """This is the front page of the Simple English Wikipedia. 
                Wikipedias are places where people work together to write encyclopedias 
                in different languages. We use Simple English words and grammar here.
                The Simple English Wikipedia is for everyone! That includes children and 
                adults who are learning English. There are 227,530 articles on the Simple 
                English Wikipedia. All of the pages are free to use."""

## Unigram language model 

Recall that the unigram language model is the simplest version of the language model, where we use the chain rule of probabilities and make a (very bad) independence assumption: we consider only the frequency (normalise this and obtain the probability) of individual words in a text and assume that each word is independent of other words (or contexts) in the sentence. 

To build a unigram model, the text is first split into individual words, and the frequency of each word is calculated. Then, the probability of each word is calculated by dividing its frequency by the total number of words in the text. This gives a probability distribution over the entire vocabulary of the language.

Given a new text, a unigram model can predict the likelihood of each word in the vocabulary based on its frequency. To generate a sentence using a unigram model, words are sampled randomly from the unigram distribution. Given sufficient amount of data this can result in sentences that are grammatically correct but may not make sense semantically.

In the code below: a sentence of length 10 is sampled from the unigram distribution using the choices() method from the random module. 

In [ ]:
import random

# Split the text into a list of words
words = input_text.split()

# Count the frequency of each word and calculate the unigram probabilities
unigram_counts = {}
for word in words:
    if word in unigram_counts:
        unigram_counts[word] += 1
    else:
        unigram_counts[word] = 1

num_words = len(words)
unigram_probs = {word: count / num_words for word, count in unigram_counts.items()}

# Sample a sentence of length 10 from the unigram distribution
sampled_sentence = []
for i in range(10):
    sampled_word = random.choices(list(unigram_probs.keys()), list(unigram_probs.values()))[0]
    sampled_sentence.append(sampled_word)

# Print out the sampled sentence
print("Sampled sentence:")
print(" ".join(sampled_sentence))

### TODO: 

1.   Try using a large sized input sample and see how the sampling changes. 
2.   Change the length of the generated samples 
3.   What happens to words that are not seen in the training set? 
4.   There is something interesting happening in the `else` loop above. What is this? Why is this done? What is the effect of not having this loop? 



## Bigram language model

Recall that the bigram language model predicts the probability of a word based on the probability of its preceding word. In other words, a bigram model considers the probability of a word based on the occurrence of its preceding word in the sentence.

Notice in the code below the bigram counts are calculated by looping over each adjacent pair of words in the text. The frequency of each bigram is stored in a dictionary called bigram_counts.

In [ ]:
# Calculate bigram counts
bigram_counts = {}
for i in range(len(words)-1):
    current_word = words[i]
    next_word = words[i+1]
    bigram = (current_word, next_word)
    if bigram in bigram_counts:
        bigram_counts[bigram] += 1
    else:
        bigram_counts[bigram] = 1

# Calculate bigram probabilities
num_bigrams = sum(bigram_counts.values())
bigram_probs = {bigram: count / num_bigrams for bigram, count in bigram_counts.items()}

# Sample a sentence of length 10 from the bigram distribution
start_word = random.choice(words)
sampled_sentence = [start_word]
for i in range(10):
    current_word = sampled_sentence[-1]
    possible_bigrams = [bigram for bigram in bigram_probs.keys() if bigram[0] == current_word]
    if len(possible_bigrams) > 0:
        sampled_bigram = random.choices(possible_bigrams, [bigram_probs[bigram] for bigram in possible_bigrams])[0]
        next_word = sampled_bigram[1]
    else:
        next_word = random.choice(words)
    sampled_sentence.append(next_word)

# Print out the sampled sentence
print("Sampled sentence:")
print(" ".join(sampled_sentence))


## Trigram language model

Now let's do trigram language model where a trigram model considers the probability of a word based on the occurrence of its two preceding words in the sentence.

It is very similar to the bigram language model and is an extension. It considers preceding two word context.


In [ ]:
# Calculate trigram counts
trigram_counts = {}
for i in range(len(words)-2):
    current_word = words[i]
    next_word = words[i+1]
    third_word = words[i+2]
    trigram = (current_word, next_word, third_word)
    if trigram in trigram_counts:
        trigram_counts[trigram] += 1
    else:
        trigram_counts[trigram] = 1

# Calculate trigram probabilities
num_trigrams = sum(trigram_counts.values())
trigram_probs = {trigram: count / num_trigrams for trigram, count in trigram_counts.items()}

# Sample a sentence of length 10 from the trigram distribution
start_bigram = random.choices([(words[i], words[i+1]) for i in range(len(words)-1)], k=1)[0]
sampled_sentence = [start_bigram[0], start_bigram[1]]
for i in range(10):
    current_bigram = (sampled_sentence[-2], sampled_sentence[-1])
    possible_trigrams = [trigram for trigram in trigram_probs.keys() if trigram[:2] == current_bigram]
    if len(possible_trigrams) > 0:
        sampled_trigram = random.choices(possible_trigrams, [trigram_probs[trigram] for trigram in possible_trigrams])[0]
        next_word = sampled_trigram[2]
    else:
        next_word = random.choice(words)
    sampled_sentence.append(next_word)

# Print out the sampled sentence
print("Sampled sentence:")
print(" ".join(sampled_sentence))

### TODO: 

1.   Try both the bigram and the trigram models with the larger dataset, do you see a difference in the generated samples? What is happening? Is the quality improving? Why is this happening?   
2.   (Challenge excercise) Extend this for 4-gram language model. What does this involve? 



## Evaluating language models: Perplexity

Recall that perplexity is a measure of how well a language model can predict a sequence of words. It is calculated as the inverse probability of the test set, normalized by the number of words in the test set, it is given as: $ppl(s) = 2^{-\scriptscriptstyle{\frac{\log P(s)}{N}}}$,

 where $s$ is a sentence, $P(s)$ is the probability of the sentence according to the LM. 

 We will define this below. 

In [ ]:
import math

def calculate_perplexity(test_set, ngram_probs):
    # Split the test set into a list of words
    test_words = test_set.split()
    
    # Calculate the perplexity of the test set
    N = len(test_words)
    log_prob = 0
    for i in range(len(test_words)-(len(ngram_probs)-1)):
        ngram = tuple(test_words[i:i+len(ngram_probs)])
        if ngram in ngram_probs:
            log_prob += math.log(ngram_probs[ngram])
        else:
            # Add probability of unknown word
            log_prob += math.log(1 / (sum(ngram_probs.values()) + 1))
    perplexity = math.exp(-log_prob/N)
    
    return perplexity

This code calculates the perplexity of a test set using a trigram language model. The test set is defined as the string "this article is from Wikipedia". Remember, lower the perplexity -- better the quality of the LM (when trained on language related data). 

Another way to think of perplexity is essentially asking to what extent is the LM surprised with a given sample of data. 


In [ ]:
# Define the test set as a string
test_set = "this article is from Wikipedia"

# Calculate perplexity of test set using the trigram language model
trigram_perplexity = calculate_perplexity(test_set, trigram_probs)
print("Trigram perplexity:", trigram_perplexity)



### TODO

1. Can you compare the perplexity from each of the LMs that we have implemented? 

2. If you have constructed 4-gram language model, please evaluate the perplexity of the model. 

3. Can you go on in this way to {5, 6, 7, 9}-grams? Where is the bottleneck? What would extending the context window do? 